In [1]:
import geopandas as gpd
import pandas as pd
import os
import requests
from io import StringIO

In [2]:
bikeTheftData = pd.read_csv('bike_theft.csv')
bikeTheftData

,LSOA Code,LSOA Name,Borough,202210,202211,202212,202301,202302,202303,202304,...,202312,202401,202402,202403,202404,202405,202406,202407,202408,202409
0,E01000007,Barking and Dagenham 015A,E09000002,0,2,0,0,2,0,0,...,0,1,1,0,1,0,0,0,1,0
1,E01000009,Barking and Dagenham 016B,E09000002,1,0,0,1,0,0,0,...,1,1,0,0,0,0,1,0,0,0
2,E01000011,Barking and Dagenham 016C,E09000002,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,E01000012,Barking and Dagenham 015D,E09000002,0,0,0,1,0,0,2,...,0,0,0,1,0,0,1,0,0,0
4,E01000013,Barking and Dagenham 013A,E09000002,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3883,E01035718,Westminster 019G,E09000033,6,1,1,1,2,1,2,...,0,1,4,3,2,1,4,8,3,3
3884,E01035719,Westminster 021F,E09000033,0,0,1,1,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3885,E01035720,Westminster 021G,E09000033,1,0,2,0,0,0,1,...,0,1,0,0,1,1,0,0,1,2
3886,E01035721,Westminster 023H,E09000033,1,1,0,0,1,1,0,...,0,0,1,2,1,0,0,1,0,1


In [3]:
month_columns = bikeTheftData.columns[3:] 
bikeTheftData[month_columns] = bikeTheftData[month_columns].apply(pd.to_numeric, errors='coerce')
bikeTheftData['Total'] = bikeTheftData[month_columns].sum(axis=1)
NewBikeTheftData = bikeTheftData[['LSOA Code','LSOA Name','Borough','Total']]
##NewBikeTheftData.to_csv('bikeTheftDataTotal.csv', index=False)
NewBikeTheftData

,LSOA Code,LSOA Name,Borough,Total
0,E01000007,Barking and Dagenham 015A,E09000002,15
1,E01000009,Barking and Dagenham 016B,E09000002,5
2,E01000011,Barking and Dagenham 016C,E09000002,1
3,E01000012,Barking and Dagenham 015D,E09000002,5
4,E01000013,Barking and Dagenham 013A,E09000002,3
...,...,...,...,...
3883,E01035718,Westminster 019G,E09000033,69
3884,E01035719,Westminster 021F,E09000033,6
3885,E01035720,Westminster 021G,E09000033,14
3886,E01035721,Westminster 023H,E09000033,18


In [4]:
NewBikeTheftData.loc[:, 'Total'] = pd.to_numeric(NewBikeTheftData['Total'], errors='coerce')
totalCounts = NewBikeTheftData['Total'].value_counts().reset_index()
totalCounts.columns = ['Total', 'Times']  
totalCounts = totalCounts.sort_values(by='Total').reset_index(drop=True)
totalCounts.to_csv('totalCounts.csv',index=False)
totalCounts

,Total,Times
0,1,835
1,2,524
2,3,427
3,4,339
4,5,244
...,...,...
90,148,1
91,163,1
92,192,1
93,196,1


In [5]:
file_path = 'LB_shp/Barking and Dagenham.shp' 
try:
    gdf = gpd.read_file(file_path)
    print(gdf)
    print("\columns:")
    print(gdf.columns)
    
except Exception as e:
    print({e})

      lsoa21cd                   lsoa21nm   msoa21cd  \
0    E01000011  Barking and Dagenham 016C  E02000017   
1    E01000046  Barking and Dagenham 017D  E02000018   
2    E01000051  Barking and Dagenham 021D  E02000022   
3    E01000077  Barking and Dagenham 008F  E02000009   
4    E01000083  Barking and Dagenham 007F  E02000008   
..         ...                        ...        ...   
110  E01000027  Barking and Dagenham 001A  E02000002   
111  E01000039  Barking and Dagenham 003D  E02000004   
112  E01000090  Barking and Dagenham 020E  E02000021   
113  E01034478  Barking and Dagenham 019I  E02000020   
114  E01000095  Barking and Dagenham 018E  E02000019   

                     msoa21nm    lad22cd               lad22nm  \
0    Barking and Dagenham 016  E09000002  Barking and Dagenham   
1    Barking and Dagenham 017  E09000002  Barking and Dagenham   
2    Barking and Dagenham 021  E09000002  Barking and Dagenham   
3    Barking and Dagenham 008  E09000002  Barking and Dagenham 

In [6]:
csv_file = 'bikeTheftDataTotal.csv'
shape_dir = 'LB_shp/'
output_csv = 'bikeTheftDataWithGeometry.csv'

bikeTheftDataTotal = pd.read_csv(csv_file)
bikeTheftDataTotal['LSOA_Prefix'] = bikeTheftDataTotal['LSOA Name'].str.rsplit(' ', n=1).str[0]
bikeTheftDataTotal['LSOA Code'] = bikeTheftDataTotal['LSOA Code'].astype(str)

geometry_list = []

for index, row in bikeTheftDataTotal.iterrows():
    lsoa_prefix = row['LSOA_Prefix']
    lsoa_code = row['LSOA Code']
    
    matched_files = [f for f in os.listdir(shape_dir) if f.endswith('.shp') and lsoa_prefix in f]
    geometry_found = None

    for shp_file in matched_files:
        try:
            shp_path = os.path.join(shape_dir, shp_file)
            gdf = gpd.read_file(shp_path)
            gdf.columns = gdf.columns.str.lower()
            
            gdf = gdf.to_crs(epsg=4326)
            
            if 'lsoa21cd' in gdf.columns:
                gdf['lsoa21cd'] = gdf['lsoa21cd'].astype(str)
                match = gdf[gdf['lsoa21cd'] == lsoa_code]

                if not match.empty and match.iloc[0].geometry:
                    geometry_found = match.iloc[0].geometry
                    print(f"Geometry found and converted in {shp_file} for LSOA Code {lsoa_code}")
                    break
        except Exception as e:
            print(f"Error processing {shp_file}: {e}")
    
    geometry_list.append(geometry_found)
    print(f"Processed {index+1}/{len(bikeTheftDataTotal)}: Geometry {'found' if geometry_found else 'not found'}")

bikeTheftDataTotal['geometry'] = geometry_list

bikeTheftDataTotal = bikeTheftDataTotal.drop(columns=['LSOA_Prefix'])

bikeTheftDataTotal.to_csv(output_csv, index=False)
print(f"Data with geometry saved to: {output_csv}")

print("Geometry column summary:")
print(bikeTheftDataTotal['geometry'].head())
print("Number of missing geometries:", bikeTheftDataTotal['geometry'].isnull().sum())

Geometry found and converted in Barking and Dagenham.shp for LSOA Code E01000007
Processed 1/3888: Geometry found
Geometry found and converted in Barking and Dagenham.shp for LSOA Code E01000009
Processed 2/3888: Geometry found
Geometry found and converted in Barking and Dagenham.shp for LSOA Code E01000011
Processed 3/3888: Geometry found
Geometry found and converted in Barking and Dagenham.shp for LSOA Code E01000012
Processed 4/3888: Geometry found
Geometry found and converted in Barking and Dagenham.shp for LSOA Code E01000013
Processed 5/3888: Geometry found
Geometry found and converted in Barking and Dagenham.shp for LSOA Code E01000017
Processed 6/3888: Geometry found
Geometry found and converted in Barking and Dagenham.shp for LSOA Code E01000020
Processed 7/3888: Geometry found
Geometry found and converted in Barking and Dagenham.shp for LSOA Code E01000021
Processed 8/3888: Geometry found
Geometry found and converted in Barking and Dagenham.shp for LSOA Code E01000030
Process

In [11]:
input_csv = 'bikeTheftDataWithGeometry.csv' 
output_csv = 'bikeTheftDataWithGeometry(color).csv' 

bikeTheftDataWithGeometry = pd.read_csv(input_csv)

def assign_color_rgb(times):
    if times >= 300:
        return 'rgb(255, 0, 0)' 
    elif 100 <= times < 300:
        return 'rgb(255, 255, 0)' 
    else:
        return 'rgb(0, 150, 0)' 

bikeTheftDataWithGeometry['color'] = bikeTheftDataWithGeometry['Total'].apply(assign_color_rgb)

columns = list(bikeTheftDataWithGeometry.columns)
columns.remove('color')
columns.insert(columns.index('Total') + 1, 'color')  
bikeTheftDataWithGeometry = bikeTheftDataWithGeometry[columns]

bikeTheftDataWithGeometry.to_csv(output_csv, index=False)
print(f"New CSV file with RGB color column saved to: {output_csv}")

print(bikeTheftDataWithGeometry.head())

New CSV file with RGB color column saved to: bikeTheftDataWithGeometry(color).csv
   LSOA Code                  LSOA Name    Borough  Total             color  \
0  E01000007  Barking and Dagenham 015A  E09000002    105  rgb(255, 255, 0)   
1  E01000009  Barking and Dagenham 016B  E09000002     35    rgb(0, 150, 0)   
2  E01000011  Barking and Dagenham 016C  E09000002      7    rgb(0, 150, 0)   
3  E01000012  Barking and Dagenham 015D  E09000002     35    rgb(0, 150, 0)   
4  E01000013  Barking and Dagenham 013A  E09000002     21    rgb(0, 150, 0)   

                                            geometry  
0  POLYGON ((0.0776356122208071 51.5428004834797,...  
1  POLYGON ((0.0837975002438877 51.54203365900032...  
2  POLYGON ((0.0839140165805131 51.54292735147401...  
3  POLYGON ((0.0768631492681977 51.54461504182171...  
4  POLYGON ((0.1523045884418958 51.54463198027658...  


In [12]:
input_csv = 'bikeTheftDataWithGeometry(color).csv'  
output_geojson = 'bikeTheftDataWithGeometry(color).geojson' 

df = pd.read_csv(input_csv)

from shapely import wkt
df['geometry'] = df['geometry'].apply(wkt.loads)

gdf = gpd.GeoDataFrame(df, geometry='geometry')

gdf.to_file(output_geojson, driver='GeoJSON')
print(f"GeoJSON file successfully created: {output_geojson}")

print(gdf.head())

C:\Users\Qakig\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pyogrio\geopandas.py:662: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(


GeoJSON file successfully created: bikeTheftDataWithGeometry(color).geojson
   LSOA Code                  LSOA Name    Borough  Total             color  \
0  E01000007  Barking and Dagenham 015A  E09000002    105  rgb(255, 255, 0)   
1  E01000009  Barking and Dagenham 016B  E09000002     35    rgb(0, 150, 0)   
2  E01000011  Barking and Dagenham 016C  E09000002      7    rgb(0, 150, 0)   
3  E01000012  Barking and Dagenham 015D  E09000002     35    rgb(0, 150, 0)   
4  E01000013  Barking and Dagenham 013A  E09000002     21    rgb(0, 150, 0)   

                                            geometry  
0  POLYGON ((0.07764 51.5428, 0.07774 51.54279, 0...  
1  POLYGON ((0.0838 51.54203, 0.08383 51.54202, 0...  
2  POLYGON ((0.08391 51.54293, 0.08409 51.54283, ...  
3  POLYGON ((0.07686 51.54462, 0.07709 51.54444, ...  
4  POLYGON ((0.1523 51.54463, 0.15229 51.54462, 0...  
